In [ ]:
import sys
import pandas as pd
import numpy as np
import math
import random
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import operator
import time
from sklearn.datasets import load_digits
from mpl_toolkits.mplot3d import Axes3D
from fractions import Fraction
from sklearn import tree
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import kde
from sklearn.neighbors import NearestNeighbors
inf = 9999999.0
def loadDataSet(fileName, splitChar='\t'):
    dataSet = []
    with open(fileName) as fr:
         for line in fr.readlines():
            curline = line.strip().split(splitChar)     
            fltline = list(map(float, curline))
            dataSet.append(fltline)
    return dataSet


def distEclud(vecA, vecB):
    return np.sqrt(np.sum(np.power(vecA - vecB, 2))) 


def min_randCent(dataSet, k2):
    dataSet = dataSet[np.nonzero(dataSet[:, -1].A == 0)[0]]
    n =dataSet.shape[1]
    mincentroids = np.mat(np.zeros((k2,n)))  
    for j in range(n):
        minJ = min(dataSet[:,j])
        maxJ = max(dataSet[:,j])
        rangeJ = float(maxJ - minJ)
        mincentroids[:,j] = minJ + rangeJ * np.random.rand(k2, 1)
    return mincentroids


def min_kMeans(dataSet, k2, distMeans =distEclud, createCent = min_randCent):

    dataSet = dataSet[np.nonzero(dataSet[:, -1].A == 0)[0]]
    m = dataSet.shape[0]   
    minclusterAssment = np.mat(np.zeros((m,2)))
    mincentroids = createCent(dataSet, k2)
    clusterChanged = True  
    while clusterChanged:
        clusterChanged = False;
        for i in range(m): 
            minDist = inf; minIndex = -1;
            for j in range(k2):
                distJI = distMeans(mincentroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j  
            if minclusterAssment[i,0] != minIndex:
                clusterChanged = True  
            minclusterAssment[i,:] = minIndex,minDist**2   
        for cent in range(k2):   
            ptsInClust = dataSet[np.nonzero(minclusterAssment[:,0].A == cent)[0]]   
            mincentroids[cent,:] = np.mean(ptsInClust, axis = 0)  
    return mincentroids, minclusterAssment

def maj_randCent(dataSet, k1):
    dataSet = dataSet[np.nonzero(dataSet[:, -1].A == 1)[0]]
    n = dataSet.shape[1]
    majcentroids = np.mat(np.zeros((k1,n)))   
    for j in range(n):
        minJ = min(dataSet[:,j])
        maxJ = max(dataSet[:,j])
        rangeJ = float(maxJ - minJ)
        majcentroids[:,j] = minJ + rangeJ * np.random.rand(k1, 1)
    return majcentroids


def maj_kMeans(dataSet, k1, distMeans =distEclud, createCent = maj_randCent):

    dataSet = dataSet[np.nonzero(dataSet[:, -1].A == 1)[0]]
    m = dataSet.shape[0]   
    majclusterAssment = np.mat(np.zeros((m,2)))
    majcentroids = createCent(dataSet, k1)
    clusterChanged = True   
    while clusterChanged:
        clusterChanged = False;
        for i in range(m):  
            minDist = inf; minIndex = -1;
            for j in range(k1):
                distJI = distMeans(majcentroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j  
            if majclusterAssment[i,0] != minIndex:
                clusterChanged = True  
            majclusterAssment[i,:] = minIndex,minDist**2   
        for cent in range(k1):   
            ptsInClust = dataSet[np.nonzero(majclusterAssment[:,0].A == cent)[0]]   
            majcentroids[cent,:] = np.mean(ptsInClust, axis = 0) 
    return majcentroids, majclusterAssment

def irsmote(dataSet,k1,k2,p):
    X=dataSet[:,0:-1]
    y=dataSet[:,-1]
    majcentroids2, majclustAssing = maj_kMeans(dataSet,k1)
    mincentroids2, minclustAssing = min_kMeans(dataSet,k2)
    majcentroids2=np.mat(majcentroids2)
    majcentroids1 = majcentroids2[:,0:-1]                                 
    minsamples1 = X[np.nonzero(dataSet[:, -1].A == 0)[0]]
    s=minsamples1.shape[0]
    n=majcentroids1.shape[0]
    dist=np.zeros(shape=(n,s))
    for i in range(n):
        for j in range(s):
            dist[i,j] = distEclud(majcentroids1[i, :], minsamples1[j, :])             
    dist=dist.reshape(n*s,order='C')
    dist=np.sort(dist)
    mean_dist=np.mean(dist[0:p])
    min_dist1=np.mat(minclustAssing)
    min_dist2=min_dist1[:,-1]
    noise=[]
    clustAssing2=[]
    for min_dist in  min_dist2:
        if min_dist>mean_dist:
            noise=dataSet[np.nonzero(minclustAssing[:, -1].A == min_dist)[0]]
            clustAssing1=minclustAssing[np.nonzero(minclustAssing[:, -1].A == min_dist)[0]]  
    nn = NearestNeighbors(n_neighbors=5, n_jobs=1)
    nn.fit(X)
    distances, indices = nn.kneighbors(X) 
    to_remove = []
    dataSet2=dataSet
    for i in range(len(noise)):
        if (y[(np.where((dataSet==noise[i]).all(axis=1))[0])]  == 1 and
            np.sum(y[(np.where((dataSet==noise[i]).all(axis=1))[0])] ==y[indices[(np.where((dataSet==noise[i]).all(axis=1))[0])][1:]])==0):
            to_remove.append(i)
        elif (y[(np.where((dataSet==noise[i]).all(axis=1))[0])]  == 0 and
            np.sum(y[(np.where((dataSet==noise[i]).all(axis=1))[0])] ==y[indices[(np.where((dataSet==noise[i]).all(axis=1))[0])][1:]])==0):
            to_remove.append(i)
    to_remove = list(set(to_remove))
    noise1=np.delete(noise, to_remove, axis=0)
    dataSet1=np.delete(dataSet,noise1,axis=0)    
    X1=dataSet1[:,0:-1] 
    y1=dataSet1[:,-1]
    minsamples2 = X1[np.nonzero(dataSet1[:, -1].A == 0)[0]]
    majsamples = X1[np.nonzero(dataSet1[:, -1].A == 1)[0]]
    minsamples_count=minsamples2.shape[0]
    majsamples_count=majsamples.shape[0]
    synthetic_count=majsamples_count-minsamples_count
    sparsity_factors=(np.zeros((k2,1),dtype=np.float64))
    sparsity_sum = 0
    imbalance_ratio_threshold = 0.9
    for i in range(k2):    
        mincentroids3=mincentroids2[i,:]
        mincentroids4=mincentroids3[:,0:-1]
        minsamples3= dataSet1[np.nonzero(minclustAssing[:, 0].A == i)[0]]
        minsamples4=minsamples3[:,0:-1]
        c=minsamples4.shape[0]
        bandwidth1=((4/(3*c))**(1/5))*np.std(minsamples4)
        kde1= kde.KernelDensity(kernel='gaussian', bandwidth=bandwidth1).fit(minsamples4)
        density_factor1=kde1.score_samples(mincentroids4.reshape(1, -1))
        density_factor=np.exp(density_factor1)
        sparsity_factors[i] = 1 / density_factor
    sparsity_sum = sparsity_factors.sum()
    if sparsity_sum == 0:
            sparsity_sum = 1 # to avoid division by zero
            sparsity_sum = np.full(sparsity_factors.shape, sparsity_sum, np.asarray(sparsity_sum).dtype)  #生成一个i行1列的矩阵
    sampling_weights = (sparsity_factors / sparsity_sum)
    cluster_synthetic_count=synthetic_count*sampling_weights
    a = {i:2*i for i in range(k2)}
    for i in a:     #遍历聚类标签
        synthetic_sample1=np.zeros((int(cluster_synthetic_count[i]), dataSet.shape[1]-1))
        synthetic_sample2=np.zeros((int(cluster_synthetic_count[i]), dataSet.shape[1]))
        tem_sample=np.zeros((1, dataSet.shape[1]-1))
        minsamples=dataSet1[np.nonzero(minclustAssing[:, 0].A == i)[0]]    
        if minsamples.shape[0]>3:
            b = np.random.randint(0,minsamples.shape[0],2)
            x=minsamples[b[0]]
            y=minsamples[b[1]]
            for k in range(int(cluster_synthetic_count[i])):
                for j in range(minsamples.shape[1]-1):
                    dif1=y[0,j]-x[0,j]
                    gap=random.random()
                    tem_sample[0,j]=x[0,j]+gap*dif1
                    dif2=mincentroids4[0,j]-tem_sample[0,j]
                    synthetic_sample1[k,j]=tem_sample[0,j]+gap*dif2
                    synthetic_sample2=np.append(synthetic_sample1,np.zeros((synthetic_sample1.shape[0],1)), axis=1)
        a[i]=synthetic_sample2
        DataSet=np.array(dataSet1)
    for k2,synthetic_sample2 in a.items():              
        DataSet=np.append(DataSet,a[k2],axis=0)          
    return DataSet    
 
def main():
    dataSet = loadDataSet(r'C:\Users\jupyter\数据集\german.txt', splitChar=' ')
    dataSet = np.mat(dataSet)
    UPPER_BOUND, LOWER_BOUND = 0.9, 0.1
    data_x = np.array(dataSet[:, :-1], dtype=np.float32)  
    data_y = np.array(dataSet[:, -1], dtype=np.int32)  
    x_min = np.min(data_x, axis=0)  
    x_max = np.max(data_x, axis=0)  
    data_x = (data_x - x_min) / x_max * (UPPER_BOUND - LOWER_BOUND) + LOWER_BOUND   
    data_num = data_x.shape[0]
    feature_num = data_x.shape[1]
    rnd_index = np.arange(data_num)
    np.random.shuffle(rnd_index)  
    data_x = data_x[rnd_index]
    data_y = data_y[rnd_index]
    dataSet=np.append(data_x,data_y,axis=1)
    dataSet = np.mat(dataSet)
    X=dataSet[:,0:-1]
    y=dataSet[:,-1] 
    DataSet =irsmote(dataSet,k1=2,k2=2,p=70)
    print(DataSet)
    print(DataSet.shape)

    
if __name__ == '__main__':
    start = time.clock()
    main()
    end = time.clock()
    print('finish all in %s' % str(end - start))
    plt.show()